# 원, 삼각형, 사각형 구분하기 

In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf

seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 셋 생성하기  
- train 의 경우  

flow_from_directory 에서 배치사이즈를 키우면 
model fit에서 steps_per_epoch를 이에 따라서 맞춰줘야한다.  
현재 3으로 했고, 이미지가 45개라서 밑의 steps_per_epoch이 15인 것

In [5]:
from tensorflow.keras.preprocessing import image

train_datagen = image.ImageDataGenerator(rescale = 1/255.)           # 데이터 셋을 0과 1사이로 만들어주는 것

# 서브 디렉토리별로 데이터를 가져와서 
train_generator = train_datagen.flow_from_directory(
    'handwriting_shape/train', 
    target_size=(24,24),                    # 이미지의 속성 - 자세히 가서 픽셀 사이즈 확인
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


- test의 경우 

In [6]:
test_datagen = image.ImageDataGenerator(rescale = 1/255.)           # 데이터 셋을 0과 1사이로 만들어주는 것

# 서브 디렉토리별로 데이터를 가져와서 
test_generator = test_datagen.flow_from_directory(
    'handwriting_shape/test', 
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [7]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

In [8]:
train_generator.filenames[0]

'circle\\circle001.png'

### 모델 정의 / 설정 / 학습

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [11]:
model = Sequential([
    Conv2D(32,(3,3), input_shape=(24,24,3),activation = 'relu'),
    Conv2D(64,(3,3),activation='relu'),

    MaxPooling2D(),
    Flatten(),
    
    Dense(128,activation='relu'),
    Dense(3, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [13]:
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

### 모델 학습

In [15]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=50,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/50
15/15 [==============================] - 0s 27ms/step - loss: 0.9691 - accuracy: 0.5556 - val_loss: 0.5256 - val_accuracy: 1.0000
Epoch 2/50
15/15 [==============================] - 0s 9ms/step - loss: 0.1344 - accuracy: 0.9778 - val_loss: 0.1081 - val_accuracy: 0.9333
Epoch 3/50
15/15 [==============================] - 0s 9ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0500 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 9ms/step - loss: 8.5256e-04 - accuracy: 1.0000 - val_loss: 0.0414 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 10ms/step - loss: 2.9451e-04 - accuracy: 1.0000 - val_loss: 0.0339 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 10ms/step - loss: 1.6993e-04 - accuracy: 1.0000 - val_loss: 0.0354 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 10ms/step - loss: 1.0292e-04 - accuracy: 1.0000 - val_loss: 0.0338 - val_accuracy: 1.0000


### 모델 평가

In [17]:
model.evaluate(test_generator, steps=5)

5/5 [==============================] - 0s 2ms/step - loss: 0.0378 - accuracy: 1.0000


[0.03780193626880646, 1.0]